# Modélisation prédictive avec le jeu de données Titanic (Kaggle Beginners)

Ce projet vise à initier les apprenants à la résolution d’un problème réel de classification supervisée à partir de données ouvertes. À partir du jeu de données Titanic (projet débutant de Kaggle), ils construisent un modèle prédictif pour estimer la probabilité de survie des passagers, en mobilisant les étapes clés d’un pipeline de machine learning.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
# on affiche les 5 premières lignes de train.csv
trainData = pd.read_csv('train.csv')
trainData.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
# on affiche les 5 premières lignes de test.csv
testData = pd.read_csv('test.csv')
testData.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
# affichage du nombre de valeurs manquantes par colonne
trainData.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

On voit que :

**Age** > 177 valeurs manquantes

**Cabin** > 687 valeurs manquantes

In [ ]:
trainData.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
women = trainData.loc[trainData.Sex == 'female']['Survived']
rateWomen = sum(women)/len(women) * 100 # somme de tous les 1 (femmes survivantes) / nombre total de femme 1 ou 0

print(f'Pourcentage de femmes ayant survécu :  {rateWomen:.2f} %') # arrondi à 2 décimales

Pourcentage de femmes ayant survécu :  74.20 %


In [5]:
men = trainData.loc[trainData.Sex == 'male']['Survived']
rateMen = sum(men)/len(men) * 100

print(f'Pourcentage d\'hommes ayant survécu :  {rateMen:.2f} %')

Pourcentage d'hommes ayant survécu :  18.89 %


In [30]:
# création df avec les pourcentages
sex_survival_rates = pd.DataFrame({
    'Sex': ['female', 'male'],
    'Survived_Rate': [rateWomen, rateMen]
})

fig_sex_survived = px.bar(sex_survival_rates,
x='Sex',
y='Survived_Rate',
color='Sex',

title='Taux de survie par sexe',
labels={
'Survived_Rate': 'Taux de survie (%)',
'Sex': 'Sexe',
}
)

fig_sex_survived.show()

In [29]:
# on conserve uniquement les lignes avec un age renseigné
data_with_age_only = trainData.dropna(subset=['Age'])

age_survival_rate = data_with_age_only.groupby('Age')['Survived'].mean() * 100

age_survival_fig = px.line(
    x=age_survival_rate.index,
    y=age_survival_rate.values,

    title='Taux de survie par âge',
    labels={
        'x': 'Âge',
        'y': 'Taux de survie (%)'
    }
)

age_survival_fig.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# variable cible > on va prédire qui a survécu 1 ou n'a pas survécu 0
y = trainData['Survived'] 

# on définit quelles caractéristiques utiliser pour la prédiction
features = [
    'Pclass', # classe du billet/équivalent classe sociale > 1-Upper 2-Mid 3-Lower
    'Sex', # sexe du passager
    'SibSp', # nombre de siblings/spouses à bord
    'Parch' # nombre de parents/children à bord
]

X = pd.get_dummies(trainData[features])
Xtest = pd.get_dummies(testData[features])

model = RandomForestClassifier(
    n_estimators=100, # nombre d'arbres de la forêt > + = plus précis mais plus lent
    max_depth=5, # profondeur max par arbre > évite l'overfitting (quand le modèle s'adapte trop aux données d'entrainement)
    random_state=1 # comme une seed > pour pas que cela ne soit random à chaque fois
)

# entrainement du modèle > comme si on étudiait 1000 cas médicaux avec des diagnostics connus et mémorisait les patterns (si symptomes A+B alors maladie X)
model.fit(X, y)

# mise en application > on voit 500 nouveaux patients, applique les connaissances acquises et on sort un diagnostic pour chacun
predictions = model.predict(Xtest)

output = pd.DataFrame({
    'PassengerId': testData.PassengerId,
    'Survived': predictions # les prédictions 0 ou 1
})

output.to_csv('submission.csv', index=False)

print('success')

success
